# Predict general data

TODO: fattorizzare con View_MED_val_preds e inference_classification, che probabilmente hanno già tutto, e unire o integrare il codice in più presente quì che non c'è negli altri due file

In [1]:
%load_ext autoreload
%autoreload 2

import os
from pathlib import Path
import shutil
import csv
import re
from time import time
from collections import defaultdict
from datetime import datetime
import pandas as pd
import numpy as np
import torch

import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append("./missing_data")

from dataset.datasets import MedicanesClsDataset
from torch.utils.data import DataLoader
import models
from timm.models import create_model

# from mpl_toolkits.basemap import Basemap
# from dataset.build_dataset import get_files_from_folder, extract_dates_pattern_airmass_rgb_20200101_0000
# from dataset.build_dataset import load_cyclones_track_noheader, compute_pixel_scale, inside_tile, calc_tile_offsets, save_single_tile
# from dataset.build_dataset import split_into_tiles_subfolders_and_track_dates, create_and_save_tile_from_complete_df
# from dataset.build_dataset import create_final_df_csv
# from dataset.build_dataset import get_gruppi_date, create_tile_videos, group_df_by_offsets
# from medicane_utils.geo_const import latcorners, loncorners, x_center, y_center, create_basemap_obj
# from medicane_utils.load_files import load_all_images

from view_test_tiles import sub_select_frequency, expand_group, get_writer4animation, make_animation
from model_analysis import create_df_predictions, video_pred_2_img_pred

from arguments import prepare_finetuning_args, Args

from dataset.data_manager import BuildDataset, DataManager


file_master_df = "all_data_all_methods_tracks_complete_fast.csv"
input_dir="../fromgcloud/2023"
output_dir = "../airmassRGB/supervised/" 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Impossibile caricare la libreria torch dynamo


### non parto dal file di manos

In [ ]:
data = BuildDataset(type='SUPERVISED', master_df_path=file_master_df)
data.load_master_df()

# non applico il relabeling

# filtro selezionando solo il 2023
print(data.master_df.shape[0])
df_2023 = data.master_df[data.master_df.datetime > datetime(2023,9,1)].copy()

data.make_df_video(new_master_df=df_2023, output_dir=output_dir) #,  is_to_balance=True)

csv_file = "./general_inference_set.csv"
data.create_final_df_csv(output_dir, csv_file)


1413168
1)  ->
Creazione delle folder per i 168 video...	Salvati 0 file - Erano già presenti 2688 file - File totali 2688
168 video per il periodo (effettivo) da 2023-09-03 00:00:00 a 2023-09-03 18:35:00

2)  ->
Creazione delle folder per i 1728 video...	Salvati 0 file - Erano già presenti 27648 file - File totali 27648
1728 video per il periodo (effettivo) da 2023-09-03 23:25:00 a 2023-09-11 23:30:00



In [6]:
#df_2023.drop(columns=['lat','lon','x_pix','y_pix','name'], inplace=True)
#df_2023

#####  bonus -> mi guardo il video del periodo

In [ ]:

#basemap_obj = create_basemap_obj()
grouped = df_2023.groupby("path", dropna=False)
print(len(grouped))
# CANCELLARE: usare create_med_video
create_labeled_images_with_tiles(grouped, 'daniel_complete_tiles.gif', basemap_obj, 213, 196)

2545

### Inferenza

In [6]:
args = prepare_finetuning_args()
args.test_path = csv_file

test_m = DataManager(is_train=False, args=args, type_t='supervised', world_size=1, rank=0)
test_m.create_classif_dataloader(args)

Getting dataset...
DATASET length: 1896
Creo il DistributedSampler con world_size 1 e rank 0
Sampler_train = <torch.utils.data.distributed.DistributedSampler object at 0x7fbf01e51fd0>
Batch_size: 1


In [7]:
# carico il modello
get_prediction = True
args.test_mode = True
args.init_ckpt = './output/checkpoint-best-CL10.pth'

model = create_model(
    args.model,
    num_classes=args.nb_classes,
    drop_rate=0.0,
    drop_path_rate=args.drop_path,
    #attn_drop_rate=0.0,
    drop_block_rate=None,
    **args.__dict__
)

model.to(args.device)
model.eval()   


Caricamento del checkpoint da: ./output/checkpoint-best-CL10.pth
Caricato state_dict con chiave: model
Checkpoint caricato con successo!


VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv3d(3, 1408, kernel_size=(2, 14, 14), stride=(2, 14, 14))
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): ModuleList(
    (0): Block(
      (norm1): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=1408, out_features=4224, bias=False)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=1408, out_features=1408, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=1408, out_features=6144, bias=True)
        (act): GELU(approximate=none)
        (fc2): Linear(in_features=6144, out_features=1408, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
    )
    (1): Block(
      (norm1): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
  

In [8]:
# ottengo il dataframe delle predizioni

all_paths, all_preds, all_labels = get_path_pred_label(model, test_m.data_loader)
df_predictions = create_df_predictions(all_paths, all_preds, all_labels)

In [9]:
df_video_w_predictions = data.df_video.merge(df_predictions, on='path')

In [10]:
df_video_w_predictions#.columns

,video_id,tile_offset_x,tile_offset_y,path,label,start_time,end_time,orig_paths,predictions,labels
0,0,0,0,03-09-2023_0115_0_0,0,2023-09-03 00:00:00,2023-09-03 01:15:00,"[../fromgcloud/airmass_rgb_20230903_0000.png, ...",0,0
1,1,0,0,03-09-2023_0235_0_0,0,2023-09-03 01:20:00,2023-09-03 02:35:00,"[../fromgcloud/airmass_rgb_20230903_0120.png, ...",0,0
2,2,0,0,03-09-2023_0355_0_0,0,2023-09-03 02:40:00,2023-09-03 03:55:00,"[../fromgcloud/airmass_rgb_20230903_0240.png, ...",0,0
3,3,0,0,03-09-2023_0515_0_0,0,2023-09-03 04:00:00,2023-09-03 05:15:00,"[../fromgcloud/airmass_rgb_20230903_0400.png, ...",0,0
4,4,0,0,03-09-2023_0635_0_0,0,2023-09-03 05:20:00,2023-09-03 06:35:00,"[../fromgcloud/airmass_rgb_20230903_0520.png, ...",0,0
...,...,...,...,...,...,...,...,...,...,...
1891,1723,1065,196,11-09-2023_1810_1065_196,0,2023-09-11 16:55:00,2023-09-11 18:10:00,"[../fromgcloud/airmass_rgb_20230911_1655.png, ...",0,0
1892,1724,1065,196,11-09-2023_1930_1065_196,0,2023-09-11 18:15:00,2023-09-11 19:30:00,"[../fromgcloud/airmass_rgb_20230911_1815.png, ...",0,0
1893,1725,1065,196,11-09-2023_2050_1065_196,0,2023-09-11 19:35:00,2023-09-11 20:50:00,"[../fromgcloud/airmass_rgb_20230911_1935.png, ...",0,0
1894,1726,1065,196,11-09-2023_2210_1065_196,0,2023-09-11 20:55:00,2023-09-11 22:10:00,"[../fromgcloud/airmass_rgb_20230911_2055.png, ...",0,0


In [ ]:
df_mapping = video_pred_2_img_pred(df_video_w_predictions)

In [12]:
df_mapping

,path,predictions,tmp_label,tile_offset_x,tile_offset_y
0,../fromgcloud/airmass_rgb_20230903_0000.png,0,0,0,0
1,../fromgcloud/airmass_rgb_20230903_0005.png,0,0,0,0
2,../fromgcloud/airmass_rgb_20230903_0010.png,0,0,0,0
3,../fromgcloud/airmass_rgb_20230903_0015.png,0,0,0,0
4,../fromgcloud/airmass_rgb_20230903_0020.png,0,0,0,0
...,...,...,...,...,...
30331,../fromgcloud/airmass_rgb_20230911_2310.png,0,0,1065,196
30332,../fromgcloud/airmass_rgb_20230911_2315.png,0,0,1065,196
30333,../fromgcloud/airmass_rgb_20230911_2320.png,0,0,1065,196
30334,../fromgcloud/airmass_rgb_20230911_2325.png,0,0,1065,196


In [13]:
# se voglio limitare i frame da renderizzare

df_2023_20m = sub_select_frequency(df_2023)
print(df_2023_20m.shape[0])

7644


In [ ]:
# ottengo il dataframe di immagini per creare il video MED: 
df_data_merg = df_mapping.merge(df_2023_20m, on=['path', 'tile_offset_x', 'tile_offset_y'], how='left').drop(columns='label').rename(columns={'tmp_label':'label'})

In [3]:
#offsets = list(df_data_merg[['tile_offset_x','tile_offset_y']].value_counts().index.values)
#offsets

In [ ]:
df_data_merg

In [ ]:
from view_test_tiles import get_writer4animation, make_animation

ffmpegwriter=get_writer4animation('./ffmpeg-7.0.2-amd64-static:')

In [ ]:
make_animation(df_data_merg, nomefile='daniel_prediction_1.mp4', writer=ffmpegwriter)

 abbiamo 2528 gruppi


12.14 minuti
Video salvato: daniel_prediction_1.mp4


# Parto da folder di immagini

In [2]:
# Usa direttamente una folder di immagini (date nei nomi file)
# Intervallo temporale dedotto dai file presenti

manos_file = "medicane_data_input/medicanes_new_windows.csv"
input_root = Path(input_dir)
output_root = Path(output_dir)

# Se True: ogni subfolder è un input separato (CSV separati)
# Se False: tutte le subfolder vengono unite (CSV unico)
split_by_subfolder = False

def _ensure_trailing_slash(p):
    s = str(p)
    return s if s.endswith('/') else s + '/'

def build_from_folder(input_dir_images, output_dir_images, name):
    tracks_df = pd.read_csv(manos_file, parse_dates=['time', 'start_time', 'end_time'])

    data = BuildDataset(type='SUPERVISED')
    data.create_master_df(manos_file=manos_file, input_dir_images=input_dir_images, tracks_df=tracks_df)

    start_dt = data.master_df['datetime'].min()
    end_dt = data.master_df['datetime'].max()
    print(f"[{name}] Intervallo immagini: {start_dt} -> {end_dt}")

    output_dir_images = _ensure_trailing_slash(output_dir_images)
    os.makedirs(output_dir_images, exist_ok=True)

    # Usa le funzioni preesistenti per creare df_video e salvare le tile
    data.make_df_video(output_dir=output_dir_images, is_to_balance=False)

    suffix = (name or 'all').replace(' ', '_')
    csv_file = f"./general_inference_set_{suffix}.csv"
    data.create_final_df_csv(output_dir_images, csv_file)
    print(f"[{name}] CSV salvato in {csv_file} con {data.df_video.shape[0]} video")

    return {
        'name': name,
        'input_dir': input_dir_images,
        'output_dir': output_dir_images,
        'csv': csv_file,
        'builder': data,
    }

# Lista dei dataset preparati

datasets = []

if split_by_subfolder:
    subfolders = sorted([p for p in input_root.iterdir() if p.is_dir()])
    if not subfolders:
        datasets.append(build_from_folder(str(input_root), str(output_root), 'all'))
    else:
        for sub in subfolders:
            out_dir = output_root / sub.name
            datasets.append(build_from_folder(str(sub), str(out_dir), sub.name))
else:
    # merge: include anche eventuali subfolder (rglob)
    datasets.append(build_from_folder(str(input_root), str(output_root), 'all'))


1436 files loaded.
[all] Intervallo immagini: 2023-02-24 00:00:00 -> 2023-02-28 23:55:00
1)  ->
Sbilanciato, ma comunque tengo traccia dei positivi e negativi
Con cicloni: 30 	 ids: [0]
Senza cicloni: 1386

Creazione delle folder per i 1416 video...	
Salvati 0 file - Erano già presenti 22656 file - File totali 22656
1416 video per il periodo (effettivo) da 2023-02-24 00:45:00 a 2023-02-28 23:00:00

[all] CSV salvato in ./general_inference_set_all.csv con 1416 video


In [3]:
args = prepare_finetuning_args()

# carico il modello
get_prediction = True
args.test_mode = True
args.init_ckpt = './output/checkpoint-best-lr-again2.pth'

model = create_model(
    args.model,
    num_classes=args.nb_classes,
    drop_rate=0.0,
    drop_path_rate=args.drop_path,
    #attn_drop_rate=0.0,
    drop_block_rate=None,
    **args.__dict__
)

model.to(args.device)
model.eval()   

Caricamento del checkpoint da: ./output/checkpoint-best-lr-again2.pth
Caricato state_dict con chiave: model
Testa del modello rimossa dal checkpoint per evitare mismatch di numero di classi col preaddestrato.
Checkpoint caricato con successo!


VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv3d(3, 1408, kernel_size=(2, 14, 14), stride=(2, 14, 14))
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): ModuleList(
    (0): Block(
      (norm1): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=1408, out_features=4224, bias=False)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=1408, out_features=1408, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=1408, out_features=6144, bias=True)
        (act): GELU(approximate=none)
        (fc2): Linear(in_features=6144, out_features=1408, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
    )
    (1): Block(
      (norm1): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
  

In [4]:
# inferenza come inference_classification.py (salva CSV predictions)
from engine_for_finetuning import validation_one_epoch_collect

# usa gli stessi args/model caricati sopra
try:
    _ = model
except NameError:
    raise RuntimeError("Modello non trovato: esegui prima la cella di caricamento del modello")

try:
    _ = args
except NameError:
    args = prepare_finetuning_args()

# se non hai creato 'datasets' sopra, usa il csv_file corrente
if 'datasets' not in globals():
    datasets = [{'name': 'single', 'csv': csv_file}]

os.makedirs('output', exist_ok=True)

for d in datasets:
    csv_path = d.get('csv')
    name = d.get('name','dataset')
    if not csv_path:
        print(f"[{name}] csv non definito")
        continue
    print(f"[{name}] inferenza su {csv_path}")

    args.val_path = csv_path
    val_m = DataManager(is_train=False, args=args, type_t='supervised', world_size=1, rank=0, specify_data_path=args.val_path)
    val_m.create_classif_dataloader(args)

    model.eval()
    val_stats, all_paths, all_preds, all_labels = validation_one_epoch_collect(val_m.data_loader, model, args.device)

    df_predictions = create_df_predictions(all_paths, all_preds, all_labels)
    out_csv = Path('output') / f"inference_predictions_{name}.csv"
    df_predictions.to_csv(out_csv, index=False)
    print(f"[{name}] salvato {out_csv} - bal_acc={val_stats.get('bal_acc')}")


[all] inferenza su ./general_inference_set_all.csv
Getting dataset...
DATASET length: 1416
Creo il DistributedSampler con world_size 1 e rank 0
Sampler_train = <torch.utils.data.distributed.DistributedSampler object at 0x7fab5c494310>
Batch_size local: 1
Val:  [   0/1416]  eta: 22:48:04  loss: 0.6940 (0.6940)  acc: 0.0000 (0.0000)  bal_acc: 0.0000 (0.0000)  pod: 0.0000 (0.0000)  far: 1.0000 (1.0000)  time: 57.9691 (57.9691 -- 57.9691)  data: 1.7571 (1.7571 -- 1.7571)  max mem: 4528
Val:  [  10/1416]  eta: 2:11:40  loss: 0.6939 (0.6935)  acc: 0.0000 (0.1818)  bal_acc: 0.0000 (0.0909)  pod: 0.0000 (0.0000)  far: 1.0000 (0.8182)  time: 5.6188 (0.3808 -- 57.9691)  data: 0.1599 (0.0001 -- 1.7571)  max mem: 4528
Val:  [  20/1416]  eta: 1:12:42  loss: 0.6935 (0.6933)  acc: 0.0000 (0.3810)  bal_acc: 0.0000 (0.1905)  pod: 0.0000 (0.0000)  far: 1.0000 (0.6190)  time: 0.3830 (0.3808 -- 0.3862)  data: 0.0002 (0.0001 -- 0.0011)  max mem: 4528
Val:  [  30/1416]  eta: 0:51:45  loss: 0.6939 (0.6935)  

In [7]:
# Assicura che ffmpeg sia nel PATH (se hai la build locale)
ffmpeg_local = Path('./ffmpeg-7.0.2-amd64-static')
if ffmpeg_local.exists():
    os.environ['PATH'] = str(ffmpeg_local) + os.pathsep + os.environ.get('PATH', '')
    print(f"PATH aggiornato con {ffmpeg_local}")
else:
    print("ffmpeg locale non trovato: assicurati che ffmpeg sia nel PATH")


PATH aggiornato con ffmpeg-7.0.2-amd64-static


In [8]:
# Dopo aver eseguito l'inferenza (inference_classification.py) con i CSV generati sopra,
# imposta qui i percorsi dei CSV di predizione.

from dataset.build_dataset import calc_tile_offsets
from view_test_tiles import expand_group, make_animation_parallel_ffmpeg

# Mappa nome-dataset -> CSV delle predizioni (da inference_classification.py)
# Esempio: {"all": "output/inference_predictions.csv"}
preds_csv_map = {
    'all': 'output/inference_predictions.csv',
}

for d in datasets:
    preds_csv = preds_csv_map.get(d['name'])
    if not preds_csv or not Path(preds_csv).exists():
        print(f"[{d['name']}] preds csv non trovato: {preds_csv}")
        continue

    df_predictions = pd.read_csv(preds_csv)
    df_video_w_predictions = d['builder'].df_video.merge(df_predictions, on='path')

    # mappa video -> immagini
    df_mapping = video_pred_2_img_pred(df_video_w_predictions)

    # merge con master_df per avere le coordinate
    df_data_merg = df_mapping.merge(
        d['builder'].master_df,
        on=['path', 'tile_offset_x', 'tile_offset_y'],
        how='left'
    ).drop(columns='label').rename(columns={'tmp_label': 'label'})

    # espansione con tile mancanti (per coprire l'intero Mediterraneo)
    offsets = calc_tile_offsets(stride_x=213, stride_y=196)
    df_offsets = pd.DataFrame(offsets, columns=['tile_offset_x', 'tile_offset_y'])
    expanded_df = df_data_merg.groupby('path', group_keys=False).apply(
        lambda g: expand_group(g, df_offsets)
    ).reset_index(drop=True)

    expanded_df.predictions = expanded_df.predictions.astype('Int8')
    expanded_df.label = expanded_df.label.astype('Int8')

    nomefile = f"mediterraneo_{d['name']}"
    make_animation_parallel_ffmpeg(expanded_df, nomefile=nomefile)



>>> Generazione dei frame PNG in anim_frames_mediterraneo_all
 abbiamo 328 gruppi
Tutti i frame sono stati generati
0.05 minuti

>>> Creazione del video MP4 con ffmpeg...


ffmpeg version 7.0.2-static https://johnvansickle.com/ffmpeg/  Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 8 (Debian 8.3.0-6)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-debug --disable-ffplay --disable-indev=sndio --disable-outdev=sndio --cc=gcc --enable-fontconfig --enable-frei0r --enable-gnutls --enable-gmp --enable-libgme --enable-gray --enable-libaom --enable-libfribidi --enable-libass --enable-libvmaf --enable-libfreetype --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-librubberband --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libvorbis --enable-libopus --enable-libtheora --enable-libvidstab --enable-libvo-amrwbenc --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libdav1d --enable-libxvid --enable-libzvbi --enable-libzimg
  libavutil      59.  8.100 / 59.  8.100
  libavcodec     61.  3.100 / 61.  3.100
  libavformat   


Video salvato: mediterraneo_all.mp4

Cartella anim_frames_mediterraneo_all già esistente, non ricreo i frame. Controlla se il video è già stato creato.
Video già esistente: mediterraneo_all.mp4


# Parto da un dataset già salvato di cui ho il csv (e i video salvati)

### come sopra, e non devo saltare il Build_Dataset perché il master_df mi serve comunque (quello identico con cui ho salvato il csv) per generare il df_video da cui prendere gli orig_paths

In [ ]:
tracks_df_MED_CL10 = pd.read_csv("manos_CL10_pixel.csv", parse_dates=['time', 'start_time', 'end_time'])
# divido le track di Manos in train e test
cicloni_unici = tracks_df_MED_CL10.id_cyc_unico.unique()
train_p = 0.7
len_p = int(train_p*cicloni_unici.shape[0])
cicloni_unici_train = cicloni_unici[:len_p]
cicloni_unici_test = cicloni_unici[len_p:]
print(cicloni_unici_train.shape, cicloni_unici_test.shape)
tracks_df_train = tracks_df_MED_CL10[tracks_df_MED_CL10.id_cyc_unico.isin(cicloni_unici_train)]
tracks_df_test = tracks_df_MED_CL10[tracks_df_MED_CL10.id_cyc_unico.isin(cicloni_unici_test)]
print(tracks_df_train.shape, tracks_df_test.shape)


(20,) (20,)
(1580, 9) (1817, 9)


In [ ]:

train_m = BuildDataset(type='SUPERVISED')
train_m.get_data_ready(tracks_df_train, input_dir, output_dir, csv_file="train_CL10")
#test_m = BuildDataset(type='SUPERVISED')
#test_m.get_data_ready(tracks_df_test, input_dir, output_dir, csv_file="test_CL10")

In [4]:
train_m.df_video.shape

(148, 8)

In [ ]:
# realizzo il video di sole label
from view_test_tiles import filter_on_intervals
cl10_intervals = tracks_df_train.groupby('id_cyc_unico').agg({'start_time':'first', 'end_time':'first'})

filtered_df1 = filter_on_intervals(cl10_intervals, train_m.master_df)
ffmpegwriter=get_writer4animation('./ffmpeg-7.0.2-amd64-static:')
make_animation(filtered_df1, nomefile='testset_CL10_148.mp4', writer=ffmpegwriter)

 abbiamo 8064 gruppi
34.21 minuti
Video salvato: testset_CL10_148.mp4


In [21]:
from view_test_tiles import calc_tile_offsets
offsets = calc_tile_offsets(stride_x=213, stride_y=196)
offsets

[(0, 0),
 (213, 0),
 (426, 0),
 (639, 0),
 (852, 0),
 (1065, 0),
 (0, 196),
 (213, 196),
 (426, 196),
 (639, 196),
 (852, 196),
 (1065, 196)]

In [20]:
csv_file = "./train_CL10_148.csv"  # il train era venuto più piccolo del test, uso questo più piccolo come test

args = prepare_finetuning_args()
args.test_path = csv_file

test_datam = DataManager(is_train=False, args=args, type_t='supervised', world_size=1, rank=0)
test_datam.create_classif_dataloader(args)

Getting dataset...
DATASET length: 148
Creo il DistributedSampler con world_size 1 e rank 0
Sampler_train = <torch.utils.data.distributed.DistributedSampler object at 0x7f58f3b0b130>
Batch_size: 1


In [ ]:
### caricol il modello

In [ ]:
get_prediction = True
args.test_mode = True
args.init_ckpt = './output/checkpoint-best-CL10.pth'

model = create_model(
    args.model,
    num_classes=args.nb_classes,
    drop_rate=0.0,
    drop_path_rate=args.drop_path,
    #attn_drop_rate=0.0,
    drop_block_rate=None,
    **args.__dict__
)

model.to(args.device)
model.eval()   


### ottengo il dataframe delle predizioni e riporto le predizioni al master df

In [22]:
all_paths, all_preds, all_labels = get_path_pred_label(model, test_datam.data_loader)
df_predictions = create_df_predictions(all_paths, all_preds, all_labels)

In [25]:
# in r4ealtà non è un train_m ma un test_m  (TODO: cambiare nome all'istanza in modo che sia più simile alla classe)
df_video_w_predictions = train_m.df_video.merge(df_predictions, on='path')   

df_mapping = video_pred_2_img_pred(df_video_w_predictions)

# se voglio limitare i frame da renderizzare
#df_sub = sub_select_frequency(train_m.master_df)

# ottengo il dataframe di immagini per creare il video MED: 
######### ????uso df_sub quì??? (al posto di train_m.master_df)
df_data_merg = df_mapping.merge(train_m.master_df, on=['path', 'tile_offset_x', 'tile_offset_y'], how='left').drop(columns='label').rename(columns={'tmp_label':'label'})

### ora gestisco il caso in cui parto da un dataset dove non ho tutte le tile del mediterraneo (come quasi sempre nel training)

In [19]:
df_offsets = train_m.master_df[['tile_offset_x','tile_offset_y']].value_counts().reset_index().drop(columns=['count'])

In [ ]:
#expanded_df = df_data_merg.groupby('path', group_keys=False).apply(expand_group).reset_index(drop=True)

expanded_df = df_data_merg.groupby('path', group_keys=False).apply(lambda x: expand_group(x, df_offsets)).reset_index(drop=True)
expanded_df.predictions = expanded_df.predictions.astype('Int8')
expanded_df.label = expanded_df.label.astype('Int8')

In [51]:
expanded_df.predictions.astype('Int8')

0        <NA>
1        <NA>
2        <NA>
3        <NA>
4        <NA>
         ... 
19783    <NA>
19784    <NA>
19785    <NA>
19786    <NA>
19787    <NA>
Name: predictions, Length: 19788, dtype: Int8

### creazione video

In [48]:
ffmpegwriter=get_writer4animation('./ffmpeg-7.0.2-amd64-static:')
make_animation(expanded_df, nomefile='testset_CL10_148_pred_.mp4', writer=ffmpegwriter)

 abbiamo 1649 gruppi
7.09 minuti
Video salvato: testset_CL10_148_pred_.mp4
